<h1> Segmenting and Clustering Neighborhoods in Toronto - Michal </h1>
<h3><b> ALL 3 tasks in 1 notebook !!!!!!!!!!!!!!!!!!! </b><h3>

Starting with import and preparations of env

In [3]:
#!conda install -c conda-forge folium --yes
#!pip uninstall -y numpy
#!pip install numpy
#!pip install numpy==1.18
#!pip install scipy==1.1.0
#!pip install scikit-learn==0.21.3
#!pip install beautifulsoup4
#!pip install lxml
import requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
# libraries for displaying images
from IPython.display import Image
from IPython.core.display import HTML
from IPython.display import display_html
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import folium # plotting library
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

<h1>TASK 1 getting and preparing the data<h1>

Read wikipedia page and create DF

In [4]:
html_data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(html_data.text, 'html.parser')
table_contents = []
table = soup.find('table')
u = 0
for row in table.findAll('td'):
    cell = {}
    u = u+1
    if 'Not assigned' in row.span.text:
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (
            ((((row.span.text).split('(')[1]).strip(')')).replace(' /', ',')).replace(')', ' ')).strip(' ')
        table_contents.append(cell)


df = pd.DataFrame(table_contents)
df['Borough'] = df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade': 'Downtown Toronto Stn A',
                                       'East TorontoBusiness reply mail Processing Centre969 Eastern': 'East Toronto Business',
                                       'EtobicokeNorthwest': 'Etobicoke Northwest',
                                       'East YorkEast Toronto': 'East York/East Toronto',
                                       'MississaugaCanada Post Gateway Processing Centre': 'Mississauga'})
# replace the / value with ,
df.Neighborhood=df.Neighborhood.str.replace((" /"),(","))
print('size of dataframe is ', df.size)
print('shape  of dataframe is ', df.shape)
print('dimension of dataframe is ', df.ndim)


size of dataframe is  309
shape  of dataframe is  (103, 3)
dimension of dataframe is  2


<h1> Task 2 - get locations </h1>

Get the coordinations for postal codes from csv and join it with my primary DF

In [5]:
csvCoordPostalCodes = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv'
df_pc_coor = pd.read_csv(csvCoordPostalCodes)
df_joined = df.join(df_pc_coor.set_index('Postal Code'), on='PostalCode')


LEt's get only the Toronto data

In [6]:
df_Toronto = df_joined[df_joined['Borough'].str.contains('Toronto',regex=False)]
print('size of dataframe is ', df_Toronto.size)
print('shape  of dataframe is ', df_Toronto.shape)
print('dimension of dataframe is ', df_Toronto.ndim)

size of dataframe is  195
shape  of dataframe is  (39, 5)
dimension of dataframe is  2


LEt's find coordinates for Toronto, Canada and prepare the map:

In [7]:
geolocator = Nominatim(user_agent="http")
locToronto = geolocator.geocode("Toronto, Canada")
mapToronto = folium.Map(location=[locToronto.latitude, locToronto.longitude], zoom_start=12)
for lat,lng,borough,neighborhood in zip(df_Toronto['Latitude'],df_Toronto['Longitude'],df_Toronto['Borough'],df_Toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(mapToronto)
mapToronto

<h1> TASK 3 - Cluster the Neighborhoods</h1>

Let's do the k-means clustering -> k=5  the same as for New York

In [8]:
from sklearn.cluster import KMeans
kclusters=5
toronto_clustering = df_Toronto.drop(['PostalCode','Borough','Neighborhood'],1)
kmeans = KMeans(n_clusters = kclusters,random_state=0).fit(toronto_clustering)
kmeans.labels_[0:10]
df_Toronto.insert(0, 'ClusterLabels', kmeans.labels_)

Let's create a map with clusters 

In [9]:
map_clusters = folium.Map(location=[locToronto.latitude, locToronto.longitude], zoom_start=12)
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i * x) ** 2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_Toronto['Latitude'], df_Toronto['Longitude'],
                                  df_Toronto['Neighborhood'], df_Toronto['ClusterLabels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster - 1],
        fill=True,
        fill_color=rainbow[cluster - 1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters